
# 📊 Fase 3 — Visualização e Geração de Insights
**Projeto:** Mineração de Textos — Análise de Sentimentos no YouTube  
**CRISP-DM:** Data Understanding / Evaluation (visual)

Este notebook lê os resultados da **Fase 2** e produz **KPIs + gráficos interativos** (Plotly).  
Os gráficos também são exportados para **HTML**, facilitando a integração no Flask.


In [ ]:

# ============================================================
# 1) Importações e Parâmetros
# ============================================================
import os, json
import pandas as pd
from glob import glob
import plotly.express as px

INPUT_PATTERN = os.getenv("ANALYSIS_INPUT_PATTERN", "resultados/comentarios_analisados_*.csv")
EXPORT_DIR = os.getenv("EXPORT_DIR", "dashboards")

os.makedirs(EXPORT_DIR, exist_ok=True)
print("✅ Parâmetros prontos.")
print("INPUT_PATTERN:", INPUT_PATTERN)
print("EXPORT_DIR   :", EXPORT_DIR)

In [ ]:

# ============================================================
# 2) Carregar o último resultado da Fase 2
# ============================================================
def ultimo_csv(pattern: str) -> str:
    arquivos = sorted(glob(pattern))
    if not arquivos:
        raise FileNotFoundError("Nenhum resultado encontrado. Execute a Fase 2 antes (analise.ipynb).")
    return arquivos[-1]

INPUT_FILE = ultimo_csv(INPUT_PATTERN)
df = pd.read_csv(INPUT_FILE)
print(f"📥 Lido: {INPUT_FILE} | Linhas: {len(df)}")
df.head(3)

In [ ]:

# ============================================================
# 3) KPIs — métricas essenciais
# ============================================================
from collections import OrderedDict

kpi = OrderedDict()
kpi["total_comentarios"] = int(len(df))
dist = df["sentimento"].value_counts(normalize=True).reindex(["positivo","neutro","negativo"]).fillna(0.0)*100
kpi["pct_positivo"] = round(float(dist.get("positivo", 0.0)), 2)
kpi["pct_neutro"]   = round(float(dist.get("neutro", 0.0)), 2)
kpi["pct_negativo"] = round(float(dist.get("negativo", 0.0)), 2)

idiomas = df["idioma"].value_counts(normalize=True)*100
top5_idiomas = idiomas.head(5).round(2).to_dict()
kpi["top5_idiomas"] = top5_idiomas

por_video = df.groupby("video_titulo")["sentimento"].value_counts(normalize=True).unstack(fill_value=0.0)*100
por_video = por_video.reindex(columns=["positivo","neutro","negativo"]).fillna(0.0)

kpi["melhor_video_positivo_%"] = por_video["positivo"].idxmax() if not por_video.empty else None
kpi["melhor_video_positivo_val"] = round(float(por_video["positivo"].max()), 2) if not por_video.empty else None

kpi["pior_video_negativo_%"] = por_video["negativo"].idxmax() if not por_video.empty else None
kpi["pior_video_negativo_val"] = round(float(por_video["negativo"].max()), 2) if not por_video.empty else None

print("📌 KPIs")
for k,v in kpi.items():
    print(f"- {k}: {v}")
kpi

In [ ]:

# ============================================================
# 4) Gráfico 1 — Distribuição geral de sentimentos
# ============================================================
fig_sent = px.bar(
    df["sentimento"].value_counts().reset_index(),
    x="index", y="sentimento",
    labels={"index":"Sentimento","sentimento":"Quantidade"},
    title="Distribuição de Sentimentos (Geral)"
)
fig_sent.show()

sent_html = os.path.join(EXPORT_DIR, "01_distribuicao_sentimentos.html")
with open(sent_html, "w", encoding="utf-8") as f:
    f.write(fig_sent.to_html(include_plotlyjs="cdn", full_html=True))
sent_html

In [ ]:

# ============================================================
# 5) Gráfico 2 — Sentimentos por vídeo (empilhado)
# ============================================================
pv = df.groupby(["video_titulo","sentimento"]).size().reset_index(name="qtd")
fig_vid = px.bar(
    pv, x="video_titulo", y="qtd", color="sentimento",
    title="Sentimentos por Vídeo (Empilhado)",
    labels={"video_titulo":"Vídeo","qtd":"Quantidade","sentimento":"Sentimento"}
)
fig_vid.update_layout(xaxis={'categoryorder':'total descending'})
fig_vid.show()

vid_html = os.path.join(EXPORT_DIR, "02_sentimentos_por_video.html")
with open(vid_html, "w", encoding="utf-8") as f:
    f.write(fig_vid.to_html(include_plotlyjs="cdn", full_html=True))
vid_html

In [ ]:

# ============================================================
# 6) Gráfico 3 — Top 5 idiomas (percentual)
# ============================================================
idiomas_top = df["idioma"].value_counts(normalize=True).head(5)*100
df_idiomas = idiomas_top.round(2).reset_index()
df_idiomas.columns = ["idioma","percentual"]

fig_id = px.bar(
    df_idiomas, x="idioma", y="percentual",
    title="Top 5 Idiomas — Participação (%)",
    labels={"idioma":"Idioma","percentual":"% de Comentários"}
)
fig_id.show()

id_html = os.path.join(EXPORT_DIR, "03_top5_idiomas.html")
with open(id_html, "w", encoding="utf-8") as f:
    f.write(fig_id.to_html(include_plotlyjs="cdn", full_html=True))
id_html

In [ ]:

# ============================================================
# 7) Insights automáticos — texto interpretativo
# ============================================================
def gerar_insights(df, kpi):
    msgs = []
    msgs.append(f"Total de {kpi['total_comentarios']} comentários analisados.")
    msgs.append(f"Clima geral: {kpi['pct_positivo']}% positivo, {kpi['pct_neutro']}% neutro, {kpi['pct_negativo']}% negativo.")
    if kpi["melhor_video_positivo_%"]:
        msgs.append(f"Vídeo com maior positividade: “{kpi['melhor_video_positivo_%']}” ({kpi['melhor_video_positivo_val']}% positivo).")
    if kpi["pior_video_negativo_%"]:
        msgs.append(f"Vídeo com maior negatividade: “{kpi['pior_video_negativo_%']}” ({kpi['pior_video_negativo_val']}% negativo).")
    if kpi["top5_idiomas"]:
        top_id = ", ".join([f"{k}:{v:.1f}%" for k,v in kpi['top5_idiomas'].items()])
        msgs.append(f"Idiomas mais frequentes: {top_id}.")
    msgs.append("Sugestão: aprofundar os vídeos com maior negatividade e replicar padrões dos com maior positividade.")
    return msgs

insights = gerar_insights(df, kpi)
for m in insights:
    print("•", m)

# Exportar insights
ts = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
ins_json = os.path.join(EXPORT_DIR, f"insights_{ts}.json")
ins_txt = os.path.join(EXPORT_DIR, f"insights_{ts}.txt")

import json
with open(ins_json, "w", encoding="utf-8") as f:
    json.dump({"arquivo_fonte": INPUT_FILE, "kpi": kpi, "insights": insights}, f, ensure_ascii=False, indent=2)

with open(ins_txt, "w", encoding="utf-8") as f:
    f.write("\n".join(insights))

ins_json, ins_txt